In [2]:
!pip install xnetwork

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.1 MB/s eta 0:00:00
  Created wheel for xnetwork: filename=xnetwork-1.0.4-py3-none-any.whl size=6744 sha256=84f0f14c2ba4d2b88e43270733a6e6a0fa7ef1b39b883279665f03852c1bc551
  Stored in directory: /root/.cache/pip/wheels/d3/30/07/1f290745f5a91845aa34e5cc916f1054a4ea38a00e52e397cb
Successfully built xnetwork


In [4]:
from math import ceil
from tqdm import tqdm
import requests as rq
import pandas as pd
import json
import xnetwork as xnet
import igraph

In [44]:
url_cursor = 'https://api.openalex.org/works?per-page=200&filter=title_and_abstract.search:"Langmuir monolayers and Langmuir-Blodgett films"&cursor=*'
# First GET to next_cursor and results
response = rq.get(url=url_cursor)
result = response.json()
cursor = result['meta']['next_cursor']
data = result['results']

In [45]:
total = result['meta']['count']
per_page = 200
num_cursor = ceil(total/per_page)
# New URL to loop
url = 'https://api.openalex.org/works?mailto=luanft9@gmail.com&per-page=200&filter=title_and_abstract.search:"Langmuir monolayers and Langmuir-Blodgett films"'

In [46]:
# Next request
print("Requesting data:")
for n_cursor in tqdm(range(num_cursor)):
    # Making the url with the parameters and next_cursor
    url_cursor = url + f'&cursor={cursor}'

    # Next GET to next_cursor and results
    response = rq.get(url=url_cursor)
    result = response.json()
    cursor = result['meta']['next_cursor']
    data += result['results']
print("Request Complete")

Requesting data:


100%|██████████| 1/1 [00:00<00:00,  3.44it/s]

Request Complete


In [47]:
# Treat abstract
def abstract(abstract):
    try:
        # Extract values
        all_values = [values for list_ in abstract.values() for values in list_]

        # Find max value
        max_value = max(all_values)

        list_ = [None] * (max_value + 1)

        for key, values in abstract.items():
            for v in values:
                list_[v] = key

        abstract = " ".join(list_)

        return abstract
    except:
        return ''

# Treat titles
def titlex(title):
    if title == None:
        return 'No title'
    else:
        return title

In [48]:
names = []
titles = []
absts = []
urefs = []

for i in data:
    uid = i['id']
    title = i['title']
    abst = i['abstract_inverted_index']
    # refs = wos.utilities.getReferences(entry)
    ud_refs = i['referenced_works']

    names.append(uid)
    titles.append(titlex(title))
    absts.append(abstract(abst))
    urefs.append(ud_refs)

In [49]:
edges = []
names_set = set(names)
for uid, ud_refs in zip(names, urefs):
    for p in ud_refs:
        if p in names_set:
            pair = (uid, p)
            edges.append(pair)

In [50]:
net = igraph.Graph()
net.add_vertices(len(names))
net.vs['name'] = names
net.vs['title'] = titles
net.vs['abstract'] = absts
net.add_edges(edges)

In [51]:
# Saving network
xnet.igraph2xnet(g=net, fileName='net_electronic_tongue.xnet')